In [0]:
!pip install bert-tensorflow
!wget http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

     |████████████████████████████████| 71kB 2.4MB/s 
--2020-03-23 09:21:20--  http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv
Resolving qim.fs.quoracdn.net (qim.fs.quoracdn.net)... 151.101.1.2, 151.101.65.2, 151.101.129.2, ...
Connecting to qim.fs.quoracdn.net (qim.fs.quoracdn.net)|151.101.1.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58176133 (55M) [text/tab-separated-values]
Saving to: ‘quora_duplicate_questions.tsv’

quora_duplicate_que 100%[===================>]  55.48M  59.3MB/s    in 0.9s    

2020-03-23 09:21:22 (59.3 MB/s) - ‘quora_duplicate_questions.tsv’ saved [58176133/58176133]

--2020-03-23 09:21:23--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.203.128, 2607:f8b0:400c:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4

In [0]:
!pip install bert-tensorflow

In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling
import numpy as np
import tensorflow as tf
import pandas as pd
from tqdm import tqdm

[link text](https://)!pip install bert-tensorflow

In [0]:
BERT_VOCAB = 'uncased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = 'uncased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = 'uncased_L-12_H-768_A-12/bert_config.json'

tokenization.validate_case_matches_checkpoint(True, '')
tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=True)
MAX_SEQ_LENGTH = 100

In [0]:
df = pd.read_csv('quora_duplicate_questions.tsv', delimiter='\t').dropna()
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [0]:
remove_n=399990
drop_indices = np.random.choice(df.index, remove_n, replace=False)
df = df.drop(drop_indices)

In [0]:
left, right, label = df['question1'].tolist(), df['question2'].tolist(), df['is_duplicate'].tolist()

['What are some good rap songs to dance to?', 'I got job offer @ Chelmsford-Essex, London with £3764 PM pay-after tax deduction. Pls advice tentative monthly expenses for couple? & saving possible?', 'How racist is too racist?', 'Should I use Vicks Vapor Rub for a stuffy nose?', 'Where can I get super high quality photos for any photo booth services in Sydney?', 'If both the parents have blood group B+, can the child have A+?', 'What are the best schools to study Natural language Processing in India?', 'Is there any architectural complex/center/forum/hub anywhere in the world? As a third space.', 'What are the consequences of trolling?', 'Are the number of pistons equal to number of number of cylinders?', 'Is the US Constitution considered to be a living document? If so, why?', 'What are symptoms of low sperm count?', 'Walking 3 km daily for 60 days will lead to how much weight loss?', 'Where are some fish spas in Malacca / Melaka?', 'Which processor is faster and better for battery be

In [0]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
              tokens_a.pop()
        else:
              tokens_b.pop()

input_ids, input_masks, segment_ids = [], [], []

for i in tqdm(range(len(left))):
    tokens_a = tokenizer.tokenize(left[i])
    tokens_b = tokenizer.tokenize(right[i])
    _truncate_seq_pair(tokens_a, tokens_b, MAX_SEQ_LENGTH - 3)
    
    tokens = []
    segment_id = []
    tokens.append("[CLS]")
    segment_id.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_id.append(0)
    tokens.append("[SEP]")
    segment_id.append(0)
    for token in tokens_b:
        tokens.append(token)
        segment_id.append(1)
    tokens.append("[SEP]")
    segment_id.append(1)
    input_id = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_id)
    
    while len(input_id) < MAX_SEQ_LENGTH:
        input_id.append(0)
        input_mask.append(0)
        segment_id.append(0)
    
    input_ids.append(input_id)
    input_masks.append(input_mask)
    segment_ids.append(segment_id)

100%|██████████| 4297/4297 [00:02<00:00, 2055.07it/s]


In [0]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

In [0]:
epoch = 10
batch_size = 60
warmup_proportion = 0.1
num_train_steps = int(len(left) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)

In [0]:
class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 1,
    ):
        self.X = tf.placeholder(tf.int32, [None, None], name='inputx')
        self.segment_ids = tf.placeholder(tf.int32, [None, None], name='inputseg')
        self.input_masks = tf.placeholder(tf.int32, [None, None], name='inputipm')
        self.Y = tf.placeholder(tf.int32, [None], name='y')
        
        model = modeling.BertModel(
            config=bert_config,
            is_training=True,
            input_ids=self.X,
            input_mask=self.input_masks,
            token_type_ids=self.segment_ids,
            use_one_hot_embeddings=False)
        
        output_layer = model.get_pooled_output()
        self.logits = tf.layers.dense(output_layer, dimension_output)
        self.logits = tf.identity(self.logits, name = 'logits')
        
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        self.tfcost=tf.identity(self.cost, name='tfcost')
        self.optimizer = optimization.create_optimizer(self.cost, learning_rate, 
                                                       num_train_steps, num_warmup_steps, False)
        self.optimizertf=tf.identity(self.optimizer, name='tfopt')
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y, name='tfcorrpred'
        )
        
        self.pred=tf.argmax(self.logits, 1, output_type = tf.int32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        self.tfacc=tf.identity(self.accuracy, name='tfacc')


In [0]:
dimension_output = 2
learning_rate = 1

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.MODEL_VARIABLES, scope = 'bert')
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, BERT_INIT_CHKPNT)




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
from sklearn.model_selection import train_test_split

train_input_ids, test_input_ids, train_input_masks, test_input_masks, train_segment_ids, test_segment_ids, train_Y, test_Y = train_test_split(
    input_ids, input_masks, segment_ids, label, test_size = 0.2
)

In [0]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 3, 0, 0, 0

while True:
    lasttime = time.time()
    if EPOCH==1:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, len(train_input_ids), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_input_ids))
        batch_x = train_input_ids[i: index]
        print("Length of batch x is",len(batch_x))

        batch_masks = train_input_masks[i: index]
        batch_segment = train_segment_ids[i: index]
        batch_y = train_Y[i: index]
        acc, cost, _,prediction = sess.run(
            [model.accuracy, model.cost, model.optimizer,model.pred],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.segment_ids: batch_segment,
                model.input_masks: batch_masks
            },
        )

        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)
    

    train_loss /= len(train_input_ids) / batch_size
    train_acc /= len(train_input_ids) / batch_size
 

    EPOCH += 1


train minibatch loop:   0%|          | 0/58 [00:00<?, ?it/s]

Length of batch x is 60


train minibatch loop:   2%|▏         | 1/58 [01:15<1:11:47, 75.56s/it, accuracy=0.65, cost=0.632]

Prediction is [0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0
 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0]
Length of batch x is 60


train minibatch loop:   3%|▎         | 2/58 [02:23<1:08:28, 73.37s/it, accuracy=0.717, cost=12.6]

...

Prediction is [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Length of batch x is 60


train minibatch loop:  98%|█████████▊| 57/58 [1:04:17<01:07, 67.42s/it, accuracy=0.717, cost=158]

Prediction is [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Length of batch x is 17


train minibatch loop: 100%|██████████| 58/58 [1:04:38<00:00, 66.86s/it, accuracy=0.706, cost=266]

Prediction is [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
time taken: 3878.0960550308228
break epoch:1



In [0]:
saver.save(sess, os.path.join(output_path, 'bert_textsim1.ckpt'))

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
output_path="/content/drive/My Drive/BERT_textsim/"

In [0]:
    import os
    output_path="/content/drive/My Drive/BERT_textsim"
    checkpoint = tf.train.latest_checkpoint(output_path)
    print("\n\nCheckpoint = " + checkpoint + "\n\n")
    tf.reset_default_graph()
    sess = tf.Session()
#    sess = tf.InteractiveSession
    saver = tf.compat.v1.train.import_meta_graph(checkpoint + '.meta')
    saver.restore(sess, checkpoint)
    graph = tf.compat.v1.get_default_graph()
    sess.run(tf.compat.v1.global_variables_initializer())
    # get the tensors that we need
    inputx = graph.get_tensor_by_name('inputx:0')
    inputy = graph.get_tensor_by_name('y:0')
    inputsegid = graph.get_tensor_by_name('inputseg:0')
    inputipmask = graph.get_tensor_by_name('inputipm:0')
    tags_seq = graph.get_tensor_by_name('logits:0')

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/BERT_textsim/bert_textsim1.ckpt


In [0]:
import numpy
inputt_ids, inputt_masks, segmentt_ids = [], [], []
i=0

left[i]=input("Question1 enter")
right[i]=input("Question1 enter")
tokenst_a = tokenizer.tokenize(left[i])
tokenst_b = tokenizer.tokenize(right[i])
_truncate_seq_pair(tokenst_a, tokenst_b, MAX_SEQ_LENGTH - 3)
print(tokenst_a)
print(tokenst_b)    
tokenst = []
segmentt_id = []
tokenst.append("[CLS]")
segmentt_id.append(0)
for token in tokenst_a:
      tokenst.append(token)
      segmentt_id.append(0)
tokenst.append("[SEP]")
segmentt_id.append(0)
for token in tokenst_b:
      tokenst.append(token)
      segmentt_id.append(1)
tokenst.append("[SEP]")
segmentt_id.append(1)
inputt_id = tokenizer.convert_tokens_to_ids(tokenst)
inputt_mask = [1] * len(inputt_id)
    
while len(inputt_id) < MAX_SEQ_LENGTH:
      inputt_id.append(0)
      inputt_mask.append(0)
      segmentt_id.append(0)
while len(segmentt_id) < MAX_SEQ_LENGTH:
      segmentt_id.append(0)
inputt_ids.append(inputt_id)
inputt_masks.append(inputt_mask)
segmentt_ids.append(segmentt_id)

inputt_masks=numpy.reshape(inputt_masks,(1,100))
segmentt_ids=numpy.reshape(segmentt_ids,(1,100))
inputt_ids=numpy.reshape(inputt_ids,(1,100))

print("after input the values are")
print(print(list((tokenizer.convert_ids_to_tokens(inputt_ids[0])))))
print(print(list((tokenizer.convert_ids_to_tokens(segmentt_ids[0])))))

Question1 enterHow are you?
Question1 enterWhat is love?
['how', 'are', 'you', '?']
['what', 'is', 'love', '?']
after input the values are
['[CLS]', 'how', 'are', 'you', '?', '[SEP]', 'what', 'is', 'love', '?', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]

In [0]:
index_i=10
test_Y=[0,1,0,1]
x = tf.placeholder("float", None)
x = sess.run(
    tags_seq,
    feed_dict = {
        inputx: input_masks[index_i-1:index_i],
        inputsegid: segment_ids[index_i-1:index_i],
        inputipmask: input_ids[index_i-1:index_i],
        inputy: test_Y[index_i-1:index_i]
    },
)
pred=tf.argmax(x, 1)
print(list((tokenizer.convert_ids_to_tokens(input_ids[index_i-1]))))

['[CLS]', 'are', 'the', 'number', 'of', 'pistons', 'equal', 'to', 'number', 'of', 'number', 'of', 'cylinders', '?', '[SEP]', 'how', 'do', 'i', 'join', 'wi', '##ki', '##lea', '##ks', '?', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [0]:
print(x)
print("True" if (pred.eval(session=sess)==1) else "False")

[[ 0.43295348 -0.81401145]]
False


In [0]:
index_i=8
test_Y=[0,1,0,1]
x = tf.placeholder("float", None)
x = sess.run(
    tags_seq,
    feed_dict = {
        inputx: inputt_masks,
        inputsegid: segmentt_ids,
        inputipmask: inputt_ids,
        inputy: test_Y[0:1]
    },
)
pred=tf.argmax(x, 1)


['[CLS]', 'which', 'universities', 'in', 'the', 'us', 'offer', 'more', 'management', 'content', 'in', 'the', 'management', 'information', 'systems', 'curriculum', '?', '[SEP]', 'which', 'universities', 'in', 'the', 'us', 'offer', 'a', 'more', 'technical', 'content', 'in', 'the', 'management', 'information', 'systems', 'curriculum', '?', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
